# 티커, 종목코드 검색

회사 이름으로 티커나 종목코드를 추출하는 간단한 코드를 작성해봄

우선 기본적인것들을 깔아놓고 시작!

In [1]:
import requests
import pandas as pd

HEADERS = {
    'User-Agent': 'Mozilla',
    'X-Requested-With': 'XMLHttpRequest',
    'Content-Type': 'application/x-www-form-urlencoded'
}

def request_post(url: str, headers: dict, data: dict) -> requests.Response:
    try:
        res = requests.post(url, headers=headers, data=data)
    except Exception as e:
        print(e)
    else:
        return res

## 한국 주식시장

국내 코스피, 코스닥에 상장되어있는 주식은 고유의 "종목코드"가 있음(삼성전자: `005930`)

스크래핑할 수 있는 여러 곳이 있겠지만 [전자공시시스템](https://kind.co.kr)에서 데이터를 추출해보았음

예제로 [KT](https://finance.naver.com/item/main.naver?code=030200)의 종목코드를 추출해보자

In [2]:
def search_stock_code(company_name: str) -> dict:
    url = 'https://kind.krx.co.kr/common/searchcorpname.do'
    data = {
        'method': 'searchCorpNameJson',
        'searchCodeType': 'char',
        'searchCorpName': company_name,
    }
    res_data = request_post(url=url, headers=HEADERS, data=data)
    res_data = res_data.json()
    return res_data

In [3]:
company_name = 'KT'

In [4]:
res_data = search_stock_code(company_name)
res_data

[{'comAttrTpCd': '1',
  'repisusrtkornm': 'KT',
  'kiscomcd': '810193',
  'spotisutrdmkttpcd': '1',
  'isurcd': '03020',
  'fssunqno': '00190321',
  'secugrpId': 'ST',
  'repisusrtcd': 'A030200',
  'comabbrv': '케이티',
  'repisusrtcd2': '030200',
  'repisucd': 'KR7030200000',
  'liststatcd': 'Y'}]

In [5]:
first_search_res = res_data[0]
official_comp_name = first_search_res['repisusrtkornm']
comp_symbol = first_search_res['repisusrtcd2']

In [6]:
kt = dict(name=official_comp_name, stock_code=comp_symbol)
kt

{'name': 'KT', 'stock_code': '030200'}

종목코드를 알아냈으니 원하는 패키지를 사용하여 해당 주식 정보를 얻어낼 수 있음

여기선 [financialdatapy](https://github.com/choi-jiwoo/financialdatapy) 패키지를 사용하였음

In [7]:
from financialdatapy.stock import Stock

In [8]:
stock_code = kt['stock_code']
kt_data = Stock(stock_code, country_code='kor')

In [9]:
kt_data

Stock(symbol='030200', country_code='KOR')

주가 데이터

In [10]:
kt_price = kt_data.price()
kt_price

,Date,Close,Open,High,Low,Volume
0,2022-07-11,37000,36850,37100,36650,500690
1,2022-07-08,36650,36700,37200,36650,728610
2,2022-07-07,36950,36600,37000,36450,712090
3,2022-07-06,36600,37150,37350,36500,0
4,2022-07-05,37400,37450,37800,37250,10
5,2022-07-04,37150,36900,37200,36750,10
6,2022-07-01,36700,36850,37100,36450,921640
7,2022-06-30,36450,36750,36750,36250,210
8,2022-06-29,36950,37200,37300,36800,20
9,2022-06-28,37000,37450,37500,36900,20


재무제표 데이터

In [11]:
# 재무제표 원본 데이터를 추출하려면 DART API를 설정해주어야함. 자세한건 `financialdatapy` 문서를 참조
# kt_income_statement = kt_data.financials('income_statement')
# kt_income_statement
kt_std_income_statement = kt_data.standard_financials('income_statement')
kt_std_income_statement

,2021-12-31,2020-12-31,2019-12-31,2018-12-31
Total Revenue,2.489800e+13,2.391667e+13,2.434206e+13,2.346014e+13
Revenue,2.489800e+13,2.391667e+13,2.434206e+13,2.346014e+13
"Other Revenue, Total",0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
"Cost of Revenue, Total",9.884229e+12,9.655996e+12,1.018808e+13,9.441214e+12
Gross Profit,1.501378e+13,1.426067e+13,1.415398e+13,1.401893e+13
Total Operating Expenses,2.309660e+13,2.285582e+13,2.317740e+13,2.222378e+13
"Selling/General/Admin. Expenses, Total",8.834058e+12,8.542218e+12,8.596177e+12,8.312768e+12
Research & Development,1.689690e+11,1.569400e+11,1.650280e+11,1.767580e+11
Depreciation / Amortization,3.607637e+12,3.634285e+12,3.629490e+12,3.281732e+12
Interest Expense (Income) - Net Operating,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
